# Notebook to create a pedestrian model with the narrow corridor measurements
Run the first two cells only if you are on Google Colab. 

The first cell clones and installs the physics based pedestrian modeling repository from GitHub.

You will need to restart the session after running the first cell.

In [ ]:
!git clone https://github.com/c-pouw/physics-based-pedestrian-modeling.git
!pip install -e physics-based-pedestrian-modeling/

Afer restarting the session you can use the next cell to change the working directory to the repository that we just cloned.

In [ ]:
import os
os.chdir('physics-based-pedestrian-modeling/usage_notebooks')

If you run the notebook locally you can skip the first two cells. However you still need to install the package.

In [ ]:
import os
import logging
from pathlib import Path
from pprint import pformat

import matplotlib.pyplot as plt
from hydra import compose, initialize
from omegaconf import OmegaConf

from physped.omegaconf_resolvers import register_new_resolvers
from physped.io.readers import trajectory_reader
from physped.preprocessing.trajectories import preprocess_trajectories, process_slow_modes
from physped.core.functions_to_discretize_grid import learn_potential_from_trajectories
from physped.core.trajectory_simulator import simulate_trajectories
from physped.visualization.plot_trajectories import plot_trajectories

#### Configuration and setup

Steps:
- Read the configuration file for the narrow corridor
- Setup the logging
- Set the matplotlib plot tyle
- Change the working directory

In [ ]:
# Set the environment name
env_name = "single_paths"

# Set the data source. Use 4tu to read the trajectories from a remote repository.
data_source = "4tu" 

with initialize(version_base=None, config_path="../physped/conf", job_name="test_app"):
    config = compose(
        config_name="config",
        return_hydra_config=True,
        overrides=[
            f"params={env_name}", 
            f"params.data_source={data_source}",
            ],
    )
    register_new_resolvers(replace=True)

# set log level
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, force = True)

# set plot style
plt.style.use(Path.cwd().parent / "physped/conf/science_no_latex.mplstyle")

# change working directory
working_dir = config.hydra.run.dir
os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

The default model parameters are set to:
- Noise intensiy $\sigma = 0.9\, \textrm{m s}^{-3/2}$
- Relaxation time $\tau = 0.5$ s
- Integration timestep $\Delta t=\frac{1}{15}$

In [ ]:
logging.info("MODELING PARAMETERS: \n" + pformat(OmegaConf.to_container(config.params.model, resolve=True), depth=1))

#### Optional: overwriting the model parameters
Any parameter in the config can be overwritten like demonstrated in the cell below

In [ ]:
config.params.model.sigma = 0.9
config.params.model.taux = 0.5
config.params.model.tauu = 0.5
config.params.model.dt = 1/15

#### Read the raw trajectories 
In this notebook the trajectories are read from the remote repository on 4TU.

In [ ]:
trajectories = trajectory_reader[env_name](config)
logging.info('\n' + pformat(trajectories.head()))

#### Preprocess the trajectories.
Steps:
- Rename the columns
- Prune short trajectories
- Add a trajectory index
- Compute velocities in Cartesian coordinates
- Convert velocities to polar coordinates

In [ ]:
preprocessed_trajectories = preprocess_trajectories(trajectories, config=config)

#### Compute the slow dynamics.
Steps:
- Compute the slow velocities $\vec{u}_s$
- Compute the slow positions $\vec{x}_s$

In [ ]:
preprocessed_trajectories = process_slow_modes(preprocessed_trajectories, config)

#### Plot the preprocessed trajectories

In [ ]:
plot_trajectories(preprocessed_trajectories, config, "recorded")

#### Compute the piecewise potential
Steps:
- Create the lattice to discretize the slow dynamics
- Fit the actual dynamics conditioned to the slow dynamics with Gaussian distributions

In [ ]:
piecewise_potential = learn_potential_from_trajectories(preprocessed_trajectories, config)

#### Simulate new trajectories with the model
- Sample initial conditions from the trajectory measurements
- Integrate the generalized pedestrian model

The number of trajectories to simulate can be overwritten in the configuration as shown below. 

In [ ]:
config.params.simulation.ntrajs = 3
simulated_trajectories = simulate_trajectories(piecewise_potential, config, preprocessed_trajectories)

#### Plot the simulated trajectories

In [ ]:
plot_trajectories(simulated_trajectories, config, "simulated")